In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://salem:30333').getOrCreate()


2022-11-07 17:03:18,244 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
df = spark.read.parquet('/SO/posts.parquet')

2022-11-07 17:03:34,880 WARN scheduler.TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2022-11-07 17:03:49,880 WARN scheduler.TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2022-11-07 17:04:04,880 WARN scheduler.TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
2022-11-07 17:04:19,880 WARN scheduler.TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
df.select('_AcceptedAnswerId').distinct().count()

In [ ]:
df.columns

In [ ]:
accepted_answer_id_df = df.select('_AcceptedAnswerId').distinct().withColumnRenamed('_AcceptedAnswerId', 'TmpId')
accepted_answer_posts_df = df.join(accepted_answer_id_df, df['_Id'] == accepted_answer_id_df['TmpId']).limit(1000)
accepted_answer_posts_df.head(5)


In [ ]:
accepted_answer_posts_df.count()

In [ ]:
import nltk
nltk.download([
     "names",
     "stopwords",
     "movie_reviews",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
])


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

out_type = MapType(StringType(), StringType(), False)

def analyze_body(body_text):
#     tokens = nltk.word_tokenize(body_text)
#     tokens = [token for token in tokens if token.isalpha() ]
    return sia.polarity_scores(body_text)

analyze_udf = udf(lambda b: analyze_body(b), out_type)

In [ ]:
#accepted_answer_posts_df.withColumn("sentiments", analyze_udf("_Body")).head(20)
#accepted_answer_posts_df.head(20)
#accepted_answer_posts_df.head(20)
for (id, rdd) in spark._jsc.getPersistentRDDs().items():
    rdd.unpersist()
    print("Unpersisted {} rdd".format(id))
    